In [6]:
import pandas as pd
import numpy as np
import re as re
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC


driver = webdriver.Chrome()
def extract_data_villum(url):
    
    """
    This function will collect the grant information provided from the Villum foundation grant database with the url: "https://villumfonden.dk/da/simpel-side/det-har-vi-stoettet-projektoversigt-fra-1971?calcbrowser.view_name.value=bars_horizontal_year&calcbrowser.university.value=0"
    
    The paths are found using devtools, and found through the webpage.
    
    note: When the window opens, one must actively choose a web browser (eg. Ecosia) and close the
    the Cookie popup window - to start the extraction. 
    
    The information is saved in the lists of the following groups:
    title_l = []
    amount_L = []
    receiver_l = []
    project_cat1_l = []
    project_cat2_l = []
    
    and if data is not avaiable for a certain group, then "NA" is inserted. 
    
    The lists are finally zipped together in the biglist dataframe.
    
    input: target_url (https://villumfonden.dk/da/simpel-side/det-har-vi-stoettet-projektoversigt-fra-1971?calcbrowser.view_name.value=bars_horizontal_year&calcbrowser.university.value=0")
    return: Dataframe with text extracted
    """    

    title_l = []
    amount_L = []
    receiver_l = []
    project_cat1_l = []
    project_cat2_l = []
    
    
    stored_description_lack_info = []
    
    
    driver.get(url)
    driver.maximize_window() #opening full web browser window
    time.sleep(5)
    
    action = ActionChains(driver)
    
    #click on table button
    ActionChains(driver).move_to_element(driver.find_element(By.CSS_SELECTOR, "#block-b14theme-content > article > div.calc-browser > div.calc-browser-table > div > div.calc-browser-table-header > div.calc-browser-table-type > a.table-show-table")).perform()  #move to place
    time.sleep(5)
    driver.find_element(By.CSS_SELECTOR, "#block-b14theme-content > article > div.calc-browser > div.calc-browser-table > div > div.calc-browser-table-header > div.calc-browser-table-type > a.table-show-table").click()
    
    try:
        for page_num in range(1,50): #49 is full number (run again)
            """we click next page 49 times"""
            #time.sleep(5)
            
            #ActionChains(driver).move_to_element(driver.find_element(By.XPATH, "/html/body/div[2]/main/div/div/article/div[2]/div[1]/div/div/div[2]/table/tbody/tr[1]/td[1]")).perform()  #move to place
            for i in range(1,51):
                time.sleep(2)
                action.move_to_element(driver.find_element(By.XPATH, f"/html/body/div/div/main/div[2]/div[2]/div/article/div[2]/div[2]/div/div[2]/div[2]/table/tbody/tr[{i}]/td[1]/a")).perform()  #move to the line of interest
                
                #title
                find_element_function(title_l, f"/html/body/div/div/main/div[2]/div[2]/div/article/div[2]/div[2]/div/div[2]/div[2]/table/tbody/tr[{i}]/td[1]/a")
                
                #amount
                find_element_function(amount_L, f"/html/body/div/div/main/div[2]/div[2]/div/article/div[2]/div[2]/div/div[2]/div[2]/table/tbody/tr[{i}]/td[2]/span[2]")
                
                #receiver
                find_element_function(receiver_l, f"/html/body/div/div/main/div[2]/div[2]/div/article/div[2]/div[2]/div/div[2]/div[2]/table/tbody/tr[{i}]/td[3]")
                
                #project_cat1_l
                find_element_function(project_cat1_l, f"/html/body/div/div/main/div[2]/div[2]/div/article/div[2]/div[2]/div/div[2]/div[2]/table/tbody/tr[{i}]/td[4]")
                
                #project_cat2_l
                find_element_function(project_cat2_l, f"/html/body/div/div/main/div[2]/div[2]/div/article/div[2]/div[2]/div/div[2]/div[2]/table/tbody/tr[{i}]/td[5]")
                
                

                if i == 50:
                    ActionChains(driver).move_to_element(driver.find_element(By.XPATH, "/html/body/div/div/main/div[2]/div[2]/div/article/div[2]/div[2]/div/div[2]/div[3]/div[2]/a[2]")).perform()  #move to click button
                    element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div/div/main/div[2]/div[2]/div/article/div[2]/div[2]/div/div[2]/div[3]/div[2]/a[2]")))
                    element.click()
                    print("page number number:", page_num)
                    time.sleep(5)
                
                    
            
    except NoSuchElementException: #avoid programme crashing
        print("Loop ended")

 

    df_final = pd.DataFrame(list(zip(title_l, amount_L, receiver_l, project_cat1_l, project_cat2_l)), columns=["Title", 'Grant size (DKK)', 'Reciever', 'Project_category', "Project_subcategory"])

    return df_final


def find_element_function(listname, x_path):
    
    """
    helper function, that will first try to find the right element, 
    and if an error is raied, it will insert an "NA" instead
    """
    
    
    try:
        text_found = driver.find_element(By.XPATH, x_path).text
        listname.append(text_found)
    except ElementClickInterceptedException:
        text_found = "NA"
        listname.append(location)
    except StaleElementReferenceException:
        text_found = "NA"
        listname.append(location)
    except NoSuchElementException:
        text_found = "NA"
        listname.append(text_found)
        
        return

biglist = extract_data_villum("https://villumfonden.dk/da/simpel-side/det-har-vi-stoettet-projektoversigt-fra-1971?calcbrowser.view_name.value=bars_horizontal_year&calcbrowser.university.value=0")
pd.set_option('display.max_columns', None)


biglist.to_csv("villum_rawdata_mio.csv") #if there is an error downstream, it is saved here first
biglist_new = biglist
biglist_new = biglist_new.rename(columns={"Reciever": "Receiver"})
biglist_new["Grant size (DKK)"] = biglist_new["Grant size (DKK)"].str.replace(".", "", regex=False).str.replace(",", ".")
biglist_new["Grant size (DKK)"] = biglist_new["Grant size (DKK)"].str.replace(" kr", "", regex=False)

mio_entries = biglist_new["Grant size (DKK)"].str.contains("mio", na=False)
mio_entrie_print = biglist_new["Grant size (DKK)"][mio_entries]
mio_no_str = mio_entrie_print.str.replace("mio", "")
mio_no_numeric = pd.to_numeric(mio_no_str, errors='coerce')
new_mio_integers = (mio_no_numeric*1000000).astype(int)
biglist_new["Grant size (DKK)"][new_mio_integers.index] = new_mio_integers
biglist_new = biglist_new.iloc[:-1]
biglist_new
biglist_new.to_csv("villum_rawdata.csv")

page number number: 1
page number number: 2
page number number: 3
page number number: 4
page number number: 5
page number number: 6
page number number: 7
page number number: 8
page number number: 9
page number number: 10
page number number: 11
page number number: 12
page number number: 13
page number number: 14
page number number: 15
page number number: 16
page number number: 17
page number number: 18
page number number: 19
page number number: 20
page number number: 21
page number number: 22
page number number: 23
page number number: 24
page number number: 25
page number number: 26
page number number: 27
page number number: 28
page number number: 29
page number number: 30
page number number: 31
page number number: 32
page number number: 33
page number number: 34
page number number: 35
page number number: 36
page number number: 37
page number number: 38
page number number: 39
page number number: 40
page number number: 41
page number number: 42
page number number: 43
page number number: 